Code quality control
====================

Today we look at some ways we automate quality control in our packages. This includes:

1. Standardizing the format of your code
2. Standardizing compliance with [PEP8](https://peps.python.org/pep-0008/) style
3. Measuring coverage of the package
4. Ensuring these are satisfied before you commit your code to git.



In [ ]:
%%bash
# Clean up any existing files
rm -fr package black-example.py flake-example.py references.bib
pip uninstall -y s23bib



# Package setup for today

We start with a small package that just sorts a bibtex file by year. This package is missing an explicit license and readme file.



In [ ]:
%%bash
mkdir -p package/s23bib
cd package
git init



In [ ]:
%%writefile package/setup.py
from setuptools import setup

setup(name='s23bib',
      version='0.0.1',
      description='bibtex utilities',
      maintainer='John Kitchin',
      maintainer_email='jkitchin@andrew.cmu.edu',
      license='MIT',
      packages=['s23bib'],
      scripts=[],
      long_description='''A set of bibtex utilities''')



In [ ]:
%%writefile package/s23bib/utils.py
import bibtexparser

def sort_bibtex(bibfile, ascending=True, inplace=False):
    with open(bibfile) as bf:
        bd = bibtexparser.load(bf)
    entries = bd.entries
    entries.sort(key=lambda entry: int(entry['year']), reverse=not ascending)
    
    if inplace:
        db = bibtexparser.bibdatabase.BibDatabase
        db.entries = entries
        db.comments = []
        db.strings={}
        db.preambles=[]
        writer = bibtexparser.bwriter.BibTexWriter()
        with open(bibfile, 'w') as bibfile:
            bibfile.write(writer.write(db))
        
    else:
        return entries



In [ ]:
%%writefile package/s23bib/__init__.py
from .utils import sort_bibtex



In [ ]:
%%writefile package/s23bib/test_sort.py
import os
import pytest
import bibtexparser
from s23bib import sort_bibtex

bs = '''
@article{kitchin-2018-machin-learn-catal,
  author =	 {John R. Kitchin},
  title =	 {Machine Learning in Catalysis},
  journal =	 {Nature Catalysis},
  volume =	 1,
  number =	 4,
  pages =	 {230-232},
  year =	 2018,
  doi =		 {10.1038/s41929-018-0056-y},
  url =		 {https://doi.org/10.1038/s41929-018-0056-y},
  DATE_ADDED =	 {Sun Mar 3 16:40:42 2019},
}
@article{kitchin-2015-examp-effec,
  author =	 {John R. Kitchin},
  title =	 {Examples of Effective Data Sharing in Scientific Publishing},
  journal =	 {ACS Catalysis},
  volume =	 5,
  number =	 6,
  pages =	 {3894-3899},
  year =	 2015,
  doi =		 {10.1021/acscatal.5b00538},
  url =		 {https://doi.org/10.1021/acscatal.5b00538},
  DATE_ADDED =	 {Fri Jan 18 09:54:51 2019},
}'''

@pytest.fixture()
def setup():
    with open('test.bib', 'w') as f:
        f.write(bs)
    yield "setup"
    os.unlink('test.bib')
    
class TestSort:
    def test_sort(self, setup):
        entries = sort_bibtex('test.bib')
        assert [e['year'] for e in entries] == ['2015', '2018']    



In [ ]:
%%writefile references.bib
@article{kitchin-2018-machin-learn-catal,
  author =	 {John R. Kitchin},
  title =	 {Machine Learning in Catalysis},
  journal =	 {Nature Catalysis},
  volume =	 1,
  number =	 4,
  pages =	 {230-232},
  year =	 2018,
  doi =		 {10.1038/s41929-018-0056-y},
  url =		 {https://doi.org/10.1038/s41929-018-0056-y},
  DATE_ADDED =	 {Sun Mar 3 16:40:42 2019},
}
@article{kitchin-2015-examp-effec,
  author =	 {John R. Kitchin},
  title =	 {Examples of Effective Data Sharing in Scientific Publishing},
  journal =	 {ACS Catalysis},
  volume =	 5,
  number =	 6,
  pages =	 {3894-3899},
  year =	 2015,
  doi =		 {10.1021/acscatal.5b00538},
  url =		 {https://doi.org/10.1021/acscatal.5b00538},
  DATE_ADDED =	 {Fri Jan 18 09:54:51 2019},
}



## Install and test basic functionalities



In [ ]:
! pip install ./package



In [ ]:
from s23bib import sort_bibtex
sort_bibtex('references.bib')



In [ ]:
sort_bibtex('references.bib', inplace=True)



In [ ]:
! cat references.bib



You can see the file has been sorted by year.



# Code formatting

When people use different formatting styles it makes it more difficult to work as a team:

1. The code looks different to different people
2. People waste time changing the format
3. git diffs contain unimportant information

A way to manage this is to use an automatic formatter. One such tools is [black](https://github.com/psf/black). It is strongly opinionated on style. 



In [ ]:
%%writefile black-example.py
a=4
#how about this for loop
for i in range(a):
    b  =3
    print( a,b)#comment two



We can see what kinds of changes will be made first.



In [ ]:
%%bash
black black-example.py --diff



To actually make the changes, we run this command. This modifies the file.



In [ ]:
%%bash
black black-example.py



In [ ]:
!cat black-example.py



It is possible to fine-tune what black does (see https://black.readthedocs.io/en/stable/usage_and_configuration/the_basics.html#configuration-via-a-file). We will not consider this here.

You can run black on all the files in a directory with

    black package




In [ ]:
! black package



There are also options to only check the files, output diffs, etc. There is even [black-nb](https://pypi.org/project/black-nb/) for notebooks.



In [ ]:
! black -h



# Code style
Style is also important. Python is a little over 30 years old now. Over the last three decades many things have been learned about effective coding styles which are described in the [PEP8](https://peps.python.org/pep-0008/) Style guide. These guidelines are even coded into a package that can analyze your code and alert you to problems: https://flake8.pycqa.org/en/latest/ and https://pylint.pycqa.org/en/latest/. 

These packages are complementary and do slightly different things. 

## Let's start with flake8.



In [ ]:
%%writefile flake-example.py
a=4
#how about this for loop
for i in range(a):
    b  =3
    print( a,b)#comment two



In [ ]:
! flake8 flake-example.py



The output tells you all the places you need to fix the code. This has to be done manually. 

You can check that all your functions are documented. flake8 is extendable, and you just install a new package called [flake8-docstrings](https://gitlab.com/pycqa/flake8-docstrings). You can then specify a docstring style as an argument.

Here we choose the numpy docstring format, and run it on our package.



In [ ]:
! flake8 --docstring-convention numpy package



You can exclude directories, e.g. package/build.



In [ ]:
! flake8 --exclude package/build,package/s23bib/.ipynb_checkpoints  --docstring-convention numpy package



**Exercise** Take some time now to fix these issues. Run the cells above until they come out clean.



## pylint

A *linter* is used to check your code for a wide range of possible problems.


[pylint](https://pylint.pycqa.org/en/latest/) is an alternative to flake8 and often provides complementary information. It is also a tool for checking for errors, coding standards, etc. 




In [ ]:
!pylint --ignore build,.ipynb_checkpoints package



**Exercise** If there are residual issues, fix them so that the package is clean.



# Testing

Our package only has one test right now. We can run it with `pytest`.



In [ ]:
! pytest package



# Coverage

Your tests should *cover* as much of your code as possible. This can actually be measured using the [coverage](https://coverage.readthedocs.io/en/7.2.3/) package. There are two steps: running and reporting



In [ ]:
!coverage run -m pytest package/s23bib



In [ ]:
!coverage report package/s23bib/*.py



Our test does not cover all of the functionality in our function; it skips the inplace argument branch. It is not necessary to achieve 100% coverage. This is a tool to help you find areas of your code that is under-tested. That doesn't mean there is not a bug in there, but it does mean you have not tested it.



# Automating these

You may find chapter 9 (https://merely-useful.tech/py-rse/automate.html) helpful. It covers make in more depth than I do there.

It is a little tedious to run these each time. There are a few ways you could solve this. One is to simply create a file as a shell command that chains all the commands together:
    
    #!/bin/bash
    black package && flake8 --exclude package/build package && pylint --ignore build package && pytest package

Then you can run one command that will run these, and stop if any single command does not succeed. Put this into a file called run.sh, make it executable, and try it out. Here we use && to only run subsequent commands if the previous command succeeded.




In [ ]:
! black package && flake8 --exclude package/build package && pylint --ignore build package && pytest package



An alternative is to create a [makefile](https://www.gnu.org/software/make/manual/make.html). Make is a GNU program that allows you to create rules that run commands by name. The syntax in a make file is sensitive. The body /must/ be indented with tabs, and not spaces. These commands are run from the same directory as the makefile, so the paths are set accordingly.

Each section starts with a target name, then a list of commands that are indented by a tab. It must be tabs or you will get an error. You don't get a tab in jupyter lab when you press tab... you get 4 spaces. I had to copy the tab from somewhere else...

The all target lists some dependencies by target name. Each of these will be run when you run the all target.



In [ ]:
%%writefile package/makefile
black: 
	black .

flake8:
	flake8 --exclude build .
    
pylint:
	pylint --ignore build .
    
test:
	pytest .

all: black flake8 pylint



In [ ]:
%%bash
cd package
make all



You can also run individual targets.



In [ ]:
%%bash
cd package
make test



Note that make will exit if any rule exits with a non-zero status.

Make is complex, and does much more than this. It has many applications in building, compilation and installing software.



# Integration with git

Finally, we can look at how we can integrate all this with git. So far, we have manually run each command, and edited files, then run the commands again. That is a little tedious, and we can leverage some capability in git we have not talked about so far. 

Git has a notion of [hooks](https://git-scm.com/book/en/v2/Customizing-Git-Git-Hooks). These are programs that are run at different events that occur. There are many possible hooks that exist in the ~.git/hooks~ directory. 




In [ ]:
%%bash
cd package
ls .git/hooks



The one we are interested in is the pre-commit hook. This will be a program that runs before a commit is done, and the commit can only proceed if the program runs successfully. We can use our makefile for this. You can create .git/hooks/pre-commit with at least this content. You also need to add a shebang line (#!/bin/bash), and make the file executable (chmod +x .git/hooks/pre-commit).

Let's run some tests first.



In [ ]:
%%writefile package/.git/hooks/pre-commit
#!/bin/bash
echo "running precommit in `pwd`"
exit 0



In [ ]:
%%bash
chmod +x package/.git/hooks/pre-commit



Now, this will be run every time you try to commit, and you will not be able to make a commit if your tests don't pass.

Note that the pre-commit hook is run from the root of the git repository, and any paths used must be set accordingly.



In [ ]:
%%bash
cd package
git add *.py
git commit -m "adding pyfiles"



In [ ]:
%%writefile package/.git/hooks/pre-commit
#!/bin/bash
make all



In [ ]:
%%bash
chmod +x package/.git/hooks/pre-commit



In [ ]:
%%bash
cd package
git add *.py
git commit -m "adding pyfiles"



Now you have to fix the errors if you want to able to commit. This helps ensure you always submit good code. You can also integrate testing into this so you make sure your code doesn't have errors in it.



# pre-commit
There are more sophisticated approaches. [pre-commit](https://pre-commit.com/#intro) is another Python package that can help you create scripts for the pre-commit hook. To set it up you have to create a yaml config file like this. I find these tricky in general, and usually adapt them from documentation at pre-commit.



In [ ]:
%%writefile package/.pre-commit-config.yaml
repos:
  -  repo: https://github.com/psf/black
     rev: 23.3.0
     hooks:
     - id: black

  -  repo: https://github.com/pre-commit/pre-commit-hooks
     rev: v2.0.0
     hooks:
     - id: flake8
    
  - repo: local
    hooks:
    - id: pytest-check
      name: pytest-check
      stages: [commit]
      types: [python]
      entry: pytest
      language: system
      pass_filenames: false
      always_run: true



In [ ]:
%%bash
cd package
pre-commit install



In [ ]:
! cat package/.git/hooks/pre-commit



We can run the rules manually.



In [ ]:
%%bash
cd package
pre-commit run --all-files



This also runs automatically whenever git detects changes in a file that would be check.



In [ ]:
%%bash
cd package

git add makefile
git commit -m "add makefile"




There are a lot of things you can do with pre-commit (https://pre-commit.com/hooks.html). 



# Summary

As your package gets more sophisticated, and more people have to interact with it, it becomes more and more important that you follow some standards in formatting and styling. There are tools to help with auto-formatting, and style checking.

Testing is important to help verify that your package works correctly. There are tools to examine your package, and compute how much of it is covered by the tests.

Finally, we looked at integration of these tools with git via the pre-commit hook to make sure that you only commit high-quality code to the repository. This helps avoid needing multiple commits to fix formatting, style issues, and can be used to make sure your tests pass before you commit.




# In class exercise

Fix all the issues in the package so that you can commit the files, and have a clean package.

Make sure to ignore files like the build directory and .egg-info directory.

